<a href="https://colab.research.google.com/github/Ayelets/Mysterious-Disappearances/blob/master/graph_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain

#file = open('/content/out.dimacs10-polblogs', 'r')
#lines = file.read().split('\n')
#lines.pop(0)
#G = nx.parse_edgelist(lines, nodetype=int)

In [2]:
blogs_df=pd.read_csv('/content/BLOG ID.csv')

blogs_df.head()

,Id,Label,timeset,value,source
0,1,100monkeystyping.com,NaN,0,Blogarama
1,2,12thharmonic.com/wordpress,NaN,0,BlogCatalog
2,3,40ozblog.blogspot.com,NaN,0,"Blogarama,BlogCatalog"
3,4,4lina.tblog.com,NaN,0,Blogarama
4,5,750volts.blogspot.com,NaN,0,Blogarama


In [3]:
df_edges = pd.read_csv('/content/BLOG EDGES.csv')


In [4]:
df_edges.head()

,source,target,type,id,label,timeset,weight
0,267,1394,Directed,0,NaN,NaN,1
1,267,483,Directed,1,NaN,NaN,1
2,267,1051,Directed,2,NaN,NaN,1
3,904,1479,Directed,3,NaN,NaN,1
4,904,919,Directed,4,NaN,NaN,1


In [5]:
G = nx.from_pandas_edgelist(df_edges)

In [6]:
nodes_color= len(blogs_df)*["red"]
blogs_df["nodes_color"]=nodes_color
blogs_df["nodes_color"][blogs_df["value"] == 1] = "blue"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
node_attr = blogs_df.set_index('Id').to_dict('index')

In [8]:
nx.set_node_attributes(G, node_attr)

In [19]:
from bokeh.plotting import figure, from_networkx
from bokeh.io import output_file, show, output_notebook, push_notebook
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool,PanTool,WheelZoomTool)

# Show with Bokeh
plot = Plot(plot_width=1000, plot_height=700,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1),toolbar_location="right")
plot.title.text = "Political Blogs Graphs"

node_hover_tool = HoverTool(tooltips=[("Label", "@Label"), ("source", "@source")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(),WheelZoomTool(),PanTool())


graph_renderer = from_networkx(G, nx.spring_layout, scale=2.5, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=5,fill_color="nodes_color")
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

output_notebook()
handle = show(plot, notebook_handle=True)

#output_file('Political Blogs Graphs.html')

#show(plot)


**Louvain Community Detection**

In [10]:
import community as community_louvain

In [11]:
partition = community_louvain.best_partition(G)

In [12]:
nx.set_node_attributes(G,partition,"partition")

**central nodes**

In [13]:
H_0 = G.subgraph([x for x,y in G.nodes(data=True) if y['value']==0])
centrality_0 = nx.degree_centrality(H_0)
nx.set_node_attributes(H_0,centrality_0,"centrality_0")

H_1 = G.subgraph([x for x,y in G.nodes(data=True) if y['value']==1])
centrality_1 = nx.degree_centrality(H_1 )
nx.set_node_attributes(H_1,centrality_1,"centrality_1")

In [21]:
# 1% top centrality nodes for each group
a=0.01
b=0

top_centrality_0=dict(sorted(centrality_0.items(), key=lambda x: x[1], reverse=True)[int(b*len(centrality_0)):int(a*len(centrality_0))])
top_centrality_1=dict(sorted(centrality_1.items(), key=lambda x: x[1], reverse=True)[int(b*len(centrality_1)):int(a*len(centrality_1))])


dict_keys([855, 1051, 963, 1245, 1153, 1112])

In [16]:
partition_lable_0=int(np.median([G.nodes[i]['partition'] for i in top_centrality_0.keys()]))
partition_lable_1=int(np.median([G.nodes[i]['partition'] for i in top_centrality_1.keys()]))


**Predict the label of all nodes based on the label of the central nodes**

In [17]:
for key, value in partition.items():
    if value == partition_lable_0:
        partition[key] = 0
    elif value == partition_lable_1:
        partition[key] = 1

In [18]:
sum([G.nodes[i]['value']==G.nodes[i]["partition"] for i in G.nodes])/len(G.nodes)

0.9109477124183006